In [17]:
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets

pp = pprint.PrettyPrinter(depth = 4)
ee.Initialize()

In [18]:
N_coor = 37.604222
W_coor = -121.09449

In [19]:
s2sr = ee.ImageCollection("COPERNICUS/S2")

In [20]:
print(s2sr.size().getInfo())

6591086


In [21]:
sample_image = ee.Image(
    s2sr.filterDate("2017-06-01", "2017-06-20")
        .filterBounds(ee.Geometry.Point(W_coor, N_coor))
        .first()
)
band_names_original = sample_image.bandNames()
band_names_original.getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B10',
 'B11',
 'B12',
 'QA10',
 'QA20',
 'QA60']

In [22]:
s2_bands = ee.Dictionary({
    "B1": "aerosols",
    "B2": "blue",
    "B3": "green",
    "B4": "red",
    "B5": "red_edge1",
    "B6": "red_edge2",
    "B7": "red_edge3",
    "B8": "nir",
    "B8A": "red_edge4",
    "B9": "water_vapor",
    "B10": "cirrus",
    "B11": "swir1",
    "B12": "swir2",
    "QA10": "QA10",
    "QA20": "QA20",
    "QA60": "QA60"
})
band_names_new = s2_bands.values(sample_image.bandNames())
s2sr = s2sr.select(band_names_original, band_names_new)

In [23]:
sample_image = ee.Image(
    s2sr.filterDate("2017-06-01", "2017-06-20")
        .filterBounds(ee.Geometry.Point(W_coor, N_coor))
        .first()
)
pp.pprint(sample_image.getInfo())

{'bands': [{'crs': 'EPSG:32610',
            'crs_transform': [60.0, 0.0, 600000.0, 0.0, -60.0, 4200000.0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [1830, 1830],
            'id': 'aerosols'},
           {'crs': 'EPSG:32610',
            'crs_transform': [10.0, 0.0, 600000.0, 0.0, -10.0, 4200000.0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [10980, 10980],
            'id': 'blue'},
           {'crs': 'EPSG:32610',
            'crs_transform': [10.0, 0.0, 600000.0, 0.0, -10.0, 4200000.0],
            'data_type': {'max': 65535,
                          'min': 0,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': 

In [25]:
IPython.display.Image(url=thumbnail_url)

In [29]:
ndvi = sample_image.normalizedDifference(["nir", "red"])

In [32]:
ndvi

In [26]:
import ipyleaflet
map1 = ipyleaflet.Map(zoom=3, layout={"height":"400px"})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [27]:
dc.last_draw

{'type': 'Feature', 'geometry': None}

In [ ]:
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

In [ ]:
#Style the imgae.
tile_url = GetTileLayerUrl(sample_image.visualize(min = 0, max = 3000, gamma=1.5, bands = ["nir", "red", "green"]))
map1.add_layer(ipyleaflet.TileLayer(url=tile_url))

In [ ]:
x = ee.FeatureCollection("ft:1At_oxD74f9ABEqS9r3G_45F8qQBMErpO7fz4-Ias")

In [ ]:
map1.add_layer(x)

In [ ]:
basemap_to_tiles(x)